In [22]:
from pyomo.environ import *
import pandas as pd
import numpy as np

In [23]:
def read_input(filename):
    SystemDemand= pd.read_excel(filename, sheet_name = 'SystemDemand')
    NodeData = pd.read_excel(filename, sheet_name='NodeData')
    LineData = pd.read_excel(filename, sheet_name='LineData')

    return {'SystemDemand':SystemDemand, 'NodeData':NodeData, 'LineData':LineData}

In [24]:
data = read_input(r'Input_Files\InputData34.xlsx')
for i in data.keys():
    print(i, ':\n', data[i])
    print('\n')

SystemDemand :
    TIME   PD   QD
0     1  1.0  1.0
1     2  1.2  1.2
2     3  1.4  1.4
3     4  0.8  0.8
4     5  0.7  0.7
5     6  1.3  1.3


NodeData :
     NODES  Tb     PD    QD
0       1   1    0.0   0.0
1       2   0  130.3  80.7
2       3   0    0.0   0.0
3       4   0  130.3  80.7
4       5   0  130.3  80.7
5       6   0    0.0   0.0
6       7   0    0.0   0.0
7       8   0  130.3  80.7
8       9   0  130.3  80.7
9      10   0    0.0   0.0
10     11   0  130.3  80.7
11     12   0   77.6  47.6
12     13   0   40.8  25.5
13     14   0   40.8  25.5
14     15   0   40.8  25.5
15     16   0    7.6   4.2
16     17   0  130.3  80.7
17     18   0  130.3  80.7
18     19   0  130.3  80.7
19     20   0  130.3  80.7
20     21   0  130.3  80.7
21     22   0  130.3  80.7
22     23   0  130.3  80.7
23     24   0  130.3  80.7
24     25   0  130.3  80.7
25     26   0  130.3  80.7
26     27   0   77.6  48.1
27     28   0   42.5  27.2
28     29   0   42.5  27.2
29     30   0   42.5  27.2
30     

In [25]:
def optimization_model(inputData, Vmax, Vmin, Vnom):

    SystemDemand= inputData['SystemDemand']
    NodeData = inputData['NodeData']
    LineData = inputData['LineData']
    time = [SystemDemand.loc[i, 'TIME'] for i in SystemDemand.index]
    lines = {(LineData.loc[i, 'FROM'], LineData.loc[i, 'TO']) for i in LineData.index}
    buses = [NodeData.loc[i, 'NODES'] for i in NodeData.index]
    R = {(LineData.loc[i,'FROM'],LineData.loc[i,'TO']):LineData.loc[i,'R'] for i in LineData.index}
    X = {(LineData.loc[i,'FROM'],LineData.loc[i,'TO']):LineData.loc[i,'X'] for i in LineData.index}
    Tb = {buses[i]:NodeData.loc[i, 'Tb'] for i in NodeData.index}
    Pd = {(buses[i], time[k]):NodeData.loc[i, 'PD']*SystemDemand.loc[k, 'PD'] for k in SystemDemand.index for i in NodeData.index}
    Qd = {(buses[i], time[k]):NodeData.loc[i, 'QD']*SystemDemand.loc[k, 'QD'] for k in SystemDemand.index for i in NodeData.index}

    #---------------------------------------------------------------------------------------------------------
    #Define the Model
    #---------------------------------------------------------------------------------------------------------

    model = ConcreteModel()

    #---------------------------------------------------------------------------------------------------------
    #Define Sets
    #---------------------------------------------------------------------------------------------------------
    model.LINES = Set(initialize=lines)
    model.NODES = Set(initialize=buses)
    model.TIME = Set(ordered=True, initialize=time)

    #---------------------------------------------------------------------------------------------------------
    #Define Parameters
    #---------------------------------------------------------------------------------------------------------

    model.R = Param(model.LINES, initialize=R, mutable=True, within=NonNegativeReals)
    model.X = Param(model.LINES, initialize=X, mutable=True, within=NonNegativeReals)

    model.Pd = Param(model.NODES, model.TIME, initialize=Pd, mutable=True, within=Any)
    model.Qd = Param(model.NODES, model.TIME, initialize=Qd, mutable=True, within=Any)
    model.Tb = Param(model.NODES, initialize=Tb, mutable=True, within=Any)
    model.Vmax = Param(initialize=Vmax, mutable=True)
    model.Vmin = Param(initialize=Vmin, mutable=True)
    model.Vnom = Param(initialize=Vnom, mutable=True)

    #---------------------------------------------------------------------------------------------------------
    #Initialize Parameters
    #---------------------------------------------------------------------------------------------------------
    def ini_resistance(model, i,j):
        return model.R[i,j]/1000
    model.RM = Param(model.LINES, rule=ini_resistance)
    def ini_reactance(model, i,j):
        return model.X[i,j]/1000
    model.XM = Param(model.LINES, rule=ini_reactance)

    #---------------------------------------------------------------------------------------------------------
    #Define Variables
    #---------------------------------------------------------------------------------------------------------
    model.I = Var(model.LINES, model.TIME, initialize=0)
    model.P = Var(model.LINES, model.TIME, initialize=0)
    model.Q = Var(model.LINES, model.TIME, initialize=0)

    def active_supply_rule(model, n, t):
        if model.Tb[n] == 0:
            temp = 0.0
            model.Ps[n,t].fixed = True
        else:
            temp = 0.0
        return temp
    model.Ps = Var(model.NODES, model.TIME, initialize=active_supply_rule)

    def reactive_supply_rule(model, n, t):
        if model.Tb[n] == 0:
            temp = 0.0
            model.Qs[n,t].fixed = True
        else:
            temp = 0.0
        return temp
    model.Qs = Var(model.NODES, model.TIME, initialize=reactive_supply_rule)

    def ini_voltage(model, n, t):
        if model.Tb[n] == 0:
            temp = model.Vnom
            model.V[n,t].fixed = False
        else:
            temp = model.Vnom
            model.V[n,t].fixed = True
        return temp
    model.V = Var(model.NODES, model.TIME, initialize=ini_voltage)

    def act_loss(model):
        return sum(model.RM[i,j]*(model.I[i,j,t]**2) for i,j in model.LINES for t in model.TIME)
    model.obj = Objective(rule=act_loss)

    def active_power_flow_rule(model, k, t):
        # Power_in + Power_supplied = Power_out + Power_out_lost + Power_consumed
        return sum(model.P[i,j,t] for i,j in model.LINES if j == k) + model.Ps[k,t] == \
               sum(model.P[i,j,t] + model.RM[i,j]*(model.I[i,j,t]**2) for i,j in model.LINES if i == k) + model.Pd[k,t]
    model.active_power_flow = Constraint(model.NODES, model.TIME, rule=active_power_flow_rule)

    def reactive_power_flow_rule(model, k, t):
        # Power_in + Power_supplied = Power_out + Power_out_lost + Power_consumed
        return sum(model.Q[i,j,t] for i,j in model.LINES if j == k) + model.Qs[k,t] == \
               sum(model.Q[i,j,t] + model.XM[i,j]*(model.I[i,j,t]**2) for i,j in model.LINES if i == k) + model.Qd[k,t]
    model.reactive_power_flow = Constraint(model.NODES, model.TIME, rule=reactive_power_flow_rule)

    def voltage_drop_rule(model, i, j, t):
        return (model.V[i,t]**2) - (model.V[j,t]**2) - 2*(model.RM[i,j]*model.P[i,j,t] + model.XM[i,j]*model.Q[i,j,t]) - (model.RM[i,j]**2 + model.XM[i,j]**2)*model.I[i,j,t]**2 == 0
    model.voltage_drop = Constraint(model.LINES, model.TIME, rule=voltage_drop_rule)

    def define_current_rule(model, i, j, t):
        return (model.V[j,t]**2)*(model.I[i,j,t]**2) == (model.P[i,j,t]**2)+(model.Q[i,j,t]**2)
    model.define_current = Constraint(model.LINES, model.TIME, rule=define_current_rule)

    def voltage_limit_rule(model, n, t):
        return inequality(model.Vmin, model.V[n,t], model.Vmax)
    model.voltage_limit = Constraint(model.NODES, model.TIME, rule=voltage_limit_rule)

    def current_limit_rule(model, i, j, t):
        return 0 <= model.I[i,j,t]
    model.current_limit = Constraint(model.LINES, model.TIME, rule=current_limit_rule)

    return model

In [26]:
Vnom = 11/np.sqrt(3)    # phase voltage (   Vp = V_LL/sqrt(3)  )
Vmax = 1.05*Vnom
Vmin = 0.8*Vnom
model = optimization_model(data, Vmax, Vmin, Vnom)
#model.pprint()

In [27]:
solver = SolverFactory('ipopt')
solver.solve(model, tee=True)

Ipopt 3.12.13: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     2976
Number of nonzeros in inequality constraint Jacobian.:      396
Number of nonzeros in Lagrangian Hessian.............:      990

Total number of variables............................:      804
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Tot

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 1206, 'Number of variables': 804, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.12.13\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.20012974739074707}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [28]:
# CHECK LOSSES+DEMAND=SUPPLY
losses = value(model.obj)
supply = sum([sum(value(model.Ps[n, t]) for n in model.NODES) for t in model.TIME])
demand = sum([sum(value(model.Pd[n, t]) for n in model.NODES) for t in model.TIME])
print([supply, demand, losses])

[18539.039234024487, 16810.88, 1728.1592340244897]
